In [27]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import sys
import pickle

# Configuration
headers = {'User-Agent': 'DataEngineeringStudentBot/1.0'}
base_url = "https://www.staticstools.eu"
root_url = f"{base_url}/en/"
all_dataframes = {}

print("✅ Environment ready.")

✅ Environment ready.


In [28]:
print("🔍 STEP 1: Bootstrapping profile list...")

try:
    res = requests.get(root_url, headers=headers, timeout=15)
    soup = BeautifulSoup(res.text, 'html.parser')

    links = soup.select('a[href*="profile-"]')
    profile_list = []

    for link in links:
        p_path = link['href']
        # Robust Name Extraction: Get name from link text or URL slug
        p_name = link.text.strip()
        if not p_name:
            # Extracts 'shs' from '/en/profile-shs'
            p_name = p_path.split('profile-')[-1].split('/')[0].upper()

        p_url = f"{base_url}/{p_path.lstrip('/')}"

        try:
            p_res = requests.get(p_url, headers=headers, timeout=10)
            p_soup = BeautifulSoup(p_res.text, 'html.parser')
            select_tag = p_soup.find('select', {'name': 'profile'})

            if select_tag:
                s_ids = [opt['value'] for opt in select_tag.find_all(
                    'option') if opt.get('value')]
                profile_list.append({
                    'name': p_name,
                    'url': p_url,
                    'ids': s_ids,
                    'count': len(s_ids)
                })
                print(f"  📂 Found {p_name:10} | {len(s_ids):>3} sections.")
        except Exception as e:
            print(f"  ⚠️ Skipping {p_name}: {e}")

    df_profiles = pd.DataFrame(profile_list)
    total_expected = df_profiles['count'].sum()
    print(
        f"\n✅ Bootstrap complete. Total sections to scrape: {total_expected}")

except Exception as e:
    print(f"❌ Critical Error during bootstrap: {e}")

🔍 STEP 1: Bootstrapping profile list...
  📂 Found IPN        |  21 sections.
  📂 Found IPE        |  18 sections.
  📂 Found IPEA       |  18 sections.
  📂 Found IPEAA      |   9 sections.
  📂 Found IPEO       |  17 sections.
  📂 Found HE         |  41 sections.
  📂 Found HEA        |  24 sections.
  📂 Found HEAA       |  24 sections.
  📂 Found HEB        |  24 sections.
  📂 Found HEM        |  24 sections.
  📂 Found HD         |  42 sections.
  📂 Found HL         |  39 sections.
  📂 Found UPN        |  18 sections.
  📂 Found UE         |  13 sections.
  📂 Found UPE        |  14 sections.
  📂 Found UAP        |   9 sections.
  📂 Found LE         | 134 sections.
  📂 Found LU         |  88 sections.
  📂 Found T          |  10 sections.
  📂 Found SHS        | 105 sections.
  📂 Found RHS        | 112 sections.
  📂 Found CHS        | 216 sections.

✅ Bootstrap complete. Total sections to scrape: 1020


In [29]:
def get_section_data_with_meta(url):
    try:
        res = requests.get(url, headers=headers, timeout=10)
        if res.status_code != 200:
            return None, None

        soup = BeautifulSoup(res.text, 'html.parser')
        section_values = {}
        section_meta = {}

        cells = soup.find_all('td')
        for cell in cells:
            text = cell.text.strip()
            description = cell.get('title', '').strip()

            if '=' in text:
                parts = text.split('=')
                if len(parts) == 2:
                    prop_name = parts[0].strip()
                    val_raw_parts = parts[1].strip().split(' ')

                    val_str = val_raw_parts[0]
                    unit_str = val_raw_parts[1] if len(
                        val_raw_parts) > 1 else ""

                    try:
                        clean_val = val_str.replace(',', '.')
                        section_values[prop_name] = float(clean_val)
                    except:
                        section_values[prop_name] = val_str

                    section_meta[prop_name] = (description, unit_str)

        return section_values, section_meta
    except:
        return None, None


print("✅ Extraction function defined.")

✅ Extraction function defined.


In [30]:
# print(f"🚀 STEP 2: Starting Master Scrape...")
# current_count = 0

# for _, row in df_profiles.iterrows():
#     p_name = row['name']
#     s_ids = row['ids']
#     current_profile_rows = []
#     profile_type_meta = {}

#     for sid in s_ids:
#         current_count += 1
#         sys.stdout.write(
#             f"\r   [{current_count}/{total_expected}] Processing {p_name} -> {sid}...")
#         sys.stdout.flush()

#         # Fix: Ensure the slug is lowercase for the URL
#         family_slug = p_name.lower()
#         detail_url = f"{base_url}/en/profile-{family_slug}/{sid}/mm/show"

#         data, meta = get_section_data_with_meta(detail_url)

#         if data:
#             data['Section_ID'] = sid
#             current_profile_rows.append(data)
#             if meta:
#                 profile_type_meta.update(meta)

#         time.sleep(0.1)  # Essential to avoid being rate-limited

#     if current_profile_rows:
#         df_temp = pd.DataFrame(current_profile_rows)
#         cols = ['Section_ID'] + \
#             [c for c in df_temp.columns if c != 'Section_ID']
#         df_final = df_temp[cols].copy()

#         profile_type_meta['Section_ID'] = ('Unique Section Identifier', 'text')
#         df_final.attrs['column_meta'] = profile_type_meta

#         all_dataframes[f"df_{p_name}"] = df_final
#         print(f" ✅ {p_name} complete.")
#     else:
#         print(f" ❌ ERROR: No data found for {p_name}. Check URL: {detail_url}")

# # Save the final product
# with open('steel_profiles_full_data.pkl', 'wb') as f:
#     pickle.dump(all_dataframes, f)

# print(
#     f"\n🏆 SCRAPE COMPLETE! {len(all_dataframes)} DataFrames saved to pickle.")

In [31]:
import pickle

# Load the full dictionary with attributes preserved
with open('steel_profiles_full_data.pkl', 'rb') as f:
    all_dataframes = pickle.load(f)

print(f"✅ Data loaded successfully.")
print(f"Found {len(all_dataframes)} DataFrames in the collection.")

✅ Data loaded successfully.
Found 22 DataFrames in the collection.


In [32]:
import pandas as pd

# Load CSV without treating the first row as headers
df_rhs_csv = pd.read_csv('RHS.csv', header=None)


In [33]:
# 1. Capture and prepare metadata
current_meta = all_dataframes['df_SHS'].attrs.get('column_meta', {}).copy()

# 2. Extract the SHS dataframe
shs = all_dataframes['df_SHS'].copy()

# 3. Calculate new radii based on your logic
# r_in = t
# r_out = 1.5 * r_in (rounded to 1 decimal)
shs['r_in'] = shs['t'].astype(float)
shs['r_out'] = (shs['r_in'] * 1.5).round(1)

# 4. Drop the old 'r' column and its metadata
shs.drop(columns=['r'], inplace=True, errors='ignore')
current_meta.pop('r', None)

# 5. Add new metadata descriptions
current_meta['r_out'] = ('Radius of outer fillet', 'mm')
current_meta['r_in'] = ('Radius of inner fillet', 'mm')

# 6. Re-attach and save back to the collection
shs.attrs['column_meta'] = current_meta
all_dataframes['df_SHS'] = shs

# Quick Verification
print(f"✅ Updated df_SHS: Dropped 'r', added r_in and r_out.")
print(
    f"Sample calculation: r_in={shs.iloc[0]['r_in']}, r_out={shs.iloc[0]['r_out']}")

✅ Updated df_SHS: Dropped 'r', added r_in and r_out.
Sample calculation: r_in=2.6, r_out=3.9


In [34]:
# 1. Capture existing metadata before it's lost in the merge
current_meta = all_dataframes['df_RHS'].attrs.get('column_meta', {}).copy()

# 2. Prepare the radius data from CSV (Cols 0=ID, 5=r_out, 6=r_in)
df_radius = df_rhs_csv[[0, 5, 6]].rename(
    columns={0: 'Section_ID', 5: 'r_out', 6: 'r_in'})

# 3. Clean IDs and Merge
# We strip the '+' from Section_ID to ensure they match the CSV format
df_updated = all_dataframes['df_RHS'].copy()
df_updated['Section_ID'] = df_updated['Section_ID'].str.replace(
    '+', '', regex=False)
df_updated = df_updated.merge(df_radius, on='Section_ID', how='left')

# 4. Drop the old 'r' column and its metadata
df_updated.drop(columns=['r'], inplace=True, errors='ignore')
current_meta.pop('r', None)

# 5. Inject new metadata for the added columns
current_meta['r_out'] = ('Radius of outer fillet', 'mm')
current_meta['r_in'] = ('Radius of inner fillet', 'mm')

# 6. Re-attach the metadata to the new DataFrame and update the collection
df_updated.attrs['column_meta'] = current_meta
all_dataframes['df_RHS'] = df_updated

print("✅ Update complete: Columns added, 'r' dropped, and .attrs preserved.")

✅ Update complete: Columns added, 'r' dropped, and .attrs preserved.


In [35]:
import pandas as pd

# 1. Define the targets for suffixing
radius_targets = ['iy', 'iz', 'iu', 'iv', 'iw']

for key in list(all_dataframes.keys()):
    df = all_dataframes[key]

    # --- Part A: Handle the df_SHS 'a' -> 'h' fix ---
    if key == 'df_SHS' and 'a' in df.columns:
        # Update DataFrame
        df.rename(columns={'a': 'h'}, inplace=True)
        # Update Attributes
        if 'column_meta' in df.attrs:
            meta = df.attrs['column_meta']
            if 'a' in meta:
                meta['h'] = meta.pop('a')
        print(f"✅ Fixed {key}: Renamed 'a' to 'h'")

    # --- Part B: Suffix the Radii (iy -> iy_radius) ---
    cols_to_rename = {
        col: f"{col}_radius" for col in df.columns if col in radius_targets}

    if cols_to_rename:
        # Update DataFrame columns
        df.rename(columns=cols_to_rename, inplace=True)

        # Update Metadata Attributes keys
        if 'column_meta' in df.attrs:
            meta = df.attrs['column_meta']
            for old_col, new_col in cols_to_rename.items():
                if old_col in meta:
                    meta[new_col] = meta.pop(old_col)

        print(f"✅ Fixed {key}: Suffixed {list(cols_to_rename.keys())}")

print("\n🚀 All DataFrames and Attributes updated successfully.")

✅ Fixed df_IPN: Suffixed ['iy', 'iz', 'iw']
✅ Fixed df_IPE: Suffixed ['iy', 'iz', 'iw']
✅ Fixed df_IPEA: Suffixed ['iy', 'iz', 'iw']
✅ Fixed df_IPEAA: Suffixed ['iy', 'iz', 'iw']
✅ Fixed df_IPEO: Suffixed ['iy', 'iz', 'iw']
✅ Fixed df_HE: Suffixed ['iy', 'iz', 'iw']
✅ Fixed df_HEA: Suffixed ['iy', 'iz', 'iw']
✅ Fixed df_HEAA: Suffixed ['iy', 'iz', 'iw']
✅ Fixed df_HEB: Suffixed ['iy', 'iz', 'iw']
✅ Fixed df_HEM: Suffixed ['iy', 'iz', 'iw']
✅ Fixed df_HD: Suffixed ['iy', 'iz', 'iw']
✅ Fixed df_HL: Suffixed ['iy', 'iz', 'iw']
✅ Fixed df_UPN: Suffixed ['iy', 'iz', 'iw']
✅ Fixed df_UE: Suffixed ['iy', 'iz', 'iw']
✅ Fixed df_UPE: Suffixed ['iy', 'iz', 'iw']
✅ Fixed df_UAP: Suffixed ['iy', 'iz', 'iw']
✅ Fixed df_LE: Suffixed ['iy', 'iz', 'iu', 'iv']
✅ Fixed df_LU: Suffixed ['iy', 'iz', 'iu', 'iv']
✅ Fixed df_T: Suffixed ['iy', 'iz']
✅ Fixed df_SHS: Renamed 'a' to 'h'
✅ Fixed df_RHS: Suffixed ['iy', 'iz']

🚀 All DataFrames and Attributes updated successfully.


In [36]:
# Update the specific metadata across all tables that have these columns
for key, df in all_dataframes.items():
    if 'column_meta' in df.attrs:
        meta = df.attrs['column_meta']

        # Update iw_radius description
        if 'iw_radius' in meta:
            meta['iw_radius'] = (
                'Radius of gyration of the warping constant', 'mm')

        # Update α (alpha) unit to [deg]
        if 'α' in meta:
            desc, _ = meta['α']
            meta['α'] = (desc, 'deg')

print(
    "✅ Metadata updated: 'iw_radius' description and 'α' units [deg] are now set.")

✅ Metadata updated: 'iw_radius' description and 'α' units [deg] are now set.


In [37]:
# Create a list to store unique pairs
unique_pairs = []
# Use a set to keep track of what we've already added (name + description string)
seen_combinations = set()

for key, df in all_dataframes.items():
    if isinstance(df, pd.DataFrame) and 'column_meta' in df.attrs:
        meta = df.attrs['column_meta']
        for col in df.columns:
            desc, unit = meta.get(col, ("No description found", "N/A"))

            # Create a unique fingerprint for this specific name/desc pair
            combination = f"{col}||{desc}"

            if combination not in seen_combinations:
                unique_pairs.append({
                    'Property': col,
                    'Description': desc,
                    'Unit': unit
                })
                seen_combinations.add(combination)

# Convert to DataFrame
df_unique_pairs = pd.DataFrame(unique_pairs).sort_values(
    by=['Property', 'Description'])

print(f"📊 Unique Engineering Pairs Found: {len(df_unique_pairs)}")
print("-" * 80)
print(df_unique_pairs.to_string(index=False))

📊 Unique Engineering Pairs Found: 66
--------------------------------------------------------------------------------
  Property                                              Description   Unit
         A                                          Area of section    mm2
        AL                         Painting surface per unit lenght m2.m-1
        Ct                               Torsional modulus constant    mm3
         D                                         Depth of section     mm
         G                                     Mass per unit lenght kg.m-1
        It                                         Torsion constant    mm4
        Iu                 Second moment of area about the u-u axis    mm4
        Iv                 Second moment of area about the v-v axis    mm4
        Iw                                         Warping constant    mm6
        Iy                 Second moment of area about the y-y axis    mm4
       Iyz                                       Centrifu

In [38]:
import sqlite3
import pandas as pd

db_name = "steel_engineering_final.db"
conn = sqlite3.connect(db_name)
print(f"🚀 Final Migration: Case Sensitive + Updated Engineering Descriptions...")

all_metadata_rows = []

for key, df in all_dataframes.items():
    if key.startswith('df_'):
        table_name = key.replace('df_', 'sections_').lower()

        # 1. Cast numeric data but preserve original casing (e.g., Iy, α)
        df_sql = df.copy()
        for col in df_sql.columns:
            if col != 'Section_ID':
                df_sql[col] = pd.to_numeric(df_sql[col], errors='coerce')

        # 2. Save to SQL
        df_sql.to_sql(table_name, conn, if_exists='replace', index=False)

        # 3. Save Context-Aware Metadata
        if 'column_meta' in df.attrs:
            for col_name, (desc, unit) in df.attrs['column_meta'].items():
                all_metadata_rows.append({
                    'table_name': table_name,
                    'column_name': col_name,
                    'description': desc,
                    'unit': unit
                })
        print(f"  ✅ {table_name}: Migrated.")

# Save the master dictionary
df_dict = pd.DataFrame(all_metadata_rows)
df_dict.to_sql('data_dictionary', conn, if_exists='replace', index=False)

conn.close()
print("\n🏆 DATABASE READY: All conflicts resolved, names preserved, and descriptions updated.")

🚀 Final Migration: Case Sensitive + Updated Engineering Descriptions...


  ✅ sections_ipn: Migrated.
  ✅ sections_ipe: Migrated.
  ✅ sections_ipea: Migrated.
  ✅ sections_ipeaa: Migrated.
  ✅ sections_ipeo: Migrated.
  ✅ sections_he: Migrated.
  ✅ sections_hea: Migrated.
  ✅ sections_heaa: Migrated.
  ✅ sections_heb: Migrated.
  ✅ sections_hem: Migrated.
  ✅ sections_hd: Migrated.
  ✅ sections_hl: Migrated.
  ✅ sections_upn: Migrated.
  ✅ sections_ue: Migrated.
  ✅ sections_upe: Migrated.
  ✅ sections_uap: Migrated.
  ✅ sections_le: Migrated.
  ✅ sections_lu: Migrated.
  ✅ sections_t: Migrated.
  ✅ sections_shs: Migrated.
  ✅ sections_rhs: Migrated.
  ✅ sections_chs: Migrated.

🏆 DATABASE READY: All conflicts resolved, names preserved, and descriptions updated.


In [39]:
import sqlite3
import pandas as pd

# 1. Connect to your final database
conn = sqlite3.connect("steel_engineering_final.db")

# 2. Fetch the specific profile from the HEM table
# The Section_ID in your scrape is likely 'HE+160+M' or similar
section_query = "SELECT * FROM sections_hem WHERE Section_ID LIKE '%160%M%'"
df_values = pd.read_sql(section_query, conn).T.reset_index()
df_values.columns = ['column_name', 'Value']

# 3. Fetch the metadata from the data_dictionary for this specific table
meta_query = "SELECT column_name, description, unit FROM data_dictionary WHERE table_name = 'sections_hem'"
df_meta = pd.read_sql(meta_query, conn)

# 4. Merge values with descriptions
report = pd.merge(df_values, df_meta, on='column_name', how='left')

# Format the output for a clean display
report['Result'] = report['Value'].astype(
    str) + " " + report['unit'].fillna("")

print(f"--- 📐 FULL DATA SHEET: HE 160 M ---")
print(report[['column_name', 'description', 'Result']].to_string(index=False))

conn.close()

--- 📐 FULL DATA SHEET: HE 160 M ---
column_name                                              description             Result
 Section_ID                                Unique Section Identifier        HE160M text
          h                                         Depth of section           180.0 mm
          b                                         Width of section           166.0 mm
         tf                                         Flange thickness            23.0 mm
         tw                                            Web thickness            14.0 mm
         r1                                    Radius of root fillet            15.0 mm
         ys               Distance of centre of gravity along y-axis            83.0 mm
          d                         Depth of straight portion of web           104.0 mm
          A                                          Area of section         9705.0 mm2
         AL                         Painting surface per unit lenght        0.97 m2.